In [1]:
!pip install tensorflow keras-facenet mtcnn opencv-python


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ----------- ---------------------------- 0.5/1.9 MB 2.8 MB/s eta 0:00:01
   ---------------------------------------- 1.9/1.9 MB 5.8 MB/s eta 0:00:00
  Created wheel for keras-facenet: filename=keras_facenet-0.3.2-py3-none-any.whl size=10388 sha256=e71e55b3c81008597eac801d1c62940a7932a1a200ee4794f0d8fa7b10e490b5
  Stored in directory: c:\users\meet mansi\appdata\local\pip\cache\wheels\05\b0\f5\19ac49fedc10b1df3ee56b096edbcfa39d45794fccc6bcdbbf
Successfully built keras-facenet
  Attempting uninstall: lz4
    Found existing installation: lz4 4.3.2
    Uninstalling lz4-4.3.2:
      Successfully uninstalled lz4-4.3.2


In [2]:
from keras_facenet import FaceNet

embedder = FaceNet()
print("✅ FaceNet model loaded.")



✅ FaceNet model loaded.


In [10]:
import cv2
import numpy as np
import os

def detect_face_mtcnn(image, detector):
    results = detector.detect_faces(image)
    if results:
        x, y, w, h = results[0]['box']
        x, y = abs(x), abs(y)
        return image[y:y+h, x:x+w]
    return None

from mtcnn import MTCNN
detector = MTCNN()

known_encodings = []
known_names = []

path = r"C:\Users\Meet Mansi\Downloads\known_faces"

for filename in os.listdir(path):
    if filename.endswith(('jpg', 'jpeg', 'png')):
        img = cv2.imread(os.path.join(path, filename))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        face = detect_face_mtcnn(img_rgb, detector)

        if face is not None:
            face = cv2.resize(face, (160, 160))
            embedding = embedder.embeddings([face])[0]
            known_encodings.append(embedding)
            known_names.append(os.path.splitext(filename)[0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 314ms/step


In [11]:
from numpy.linalg import norm

def cosine_similarity(a, b):
    return np.dot(a, b) / (norm(a) * norm(b))


In [14]:
import cv2

cap = cv2.VideoCapture(0)
attendance = set()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    face = detect_face_mtcnn(rgb, detector)

    if face is not None:
        face = cv2.resize(face, (160, 160))
        embedding = embedder.embeddings([face])[0]

        for known_embed, name in zip(known_encodings, known_names):
            sim = cosine_similarity(known_embed, embedding)
            if sim > 0.6 and name not in attendance:
                attendance.add(name)
                print(f"🟢 {name} marked present")

    cv2.imshow("Webcam - Press Q to exit", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
🟢 Avi marked present
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step
🟢 Ankit marked present
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
🟢 Devanshi marked present
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 237ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms

In [13]:
print("\n✅ Final Attendance List:")
for person in attendance:
    print(f"✔️ {person}")



✅ Final Attendance List:
✔️ Avi
✔️ Ankit
✔️ kp
✔️ Shivam
✔️ Devanshi
✔️ Meet
